In [1]:
from utils.binaries import *
from utils.plotting import *
from utils.auger.fd import *
from utils.auger.fd import XYScanner
from datetime import datetime
from utils import CONSTANTS


15:39:02 (   +1.1s) [INFO   ] -- import logging
15:39:02 (  +174ms) [INFO   ] -- import numpy as np
15:39:02 (   +10ms) [INFO   ] -- import uncertainties
15:39:03 (  +384ms) [INFO   ] -- import pandas as pd
15:39:03 (    +1ms) [WARNING] -- pandas support to be dropped at some point!
15:39:03 (    +2ms) [INFO   ] -- import binaries.tools as tools
15:39:03 (  +393ms) [INFO   ] -- import matplotlib.pyplot as plt
15:39:04 (  +566ms) [INFO   ] -- import seaborn as so
15:39:04 (    +3ms) [INFO   ] -- set MONI_PATH = PosixPath('/cr/work/filip/monit_and_sd')
15:39:04 (    +1ms) [INFO   ] -- set HIST_PATH = PosixPath('/cr/work/filip/monit_and_sd')
15:39:04 (    +1ms) [INFO   ] -- set PLOT_PATH = PosixPath('/cr/data01/filip/plots')
15:39:04 (    +0ms) [INFO   ] -- set DATA_PATH = PosixPath('/cr/data01/filip/Data')
15:39:04 (    +1ms) [INFO   ] -- set SCAN_PATH = PosixPath('/cr/data01/filip/xy-calibration')
15:39:04 (    +1ms) [INFO   ] -- set OFLN_PATH = PosixPath('/cr/data01/filip/offline/insta

In [3]:
from matplotlib.backends.backend_pdf import PdfPages

campaign_dates = ["2022-10", "2023-10", "2023-11", "2024-11"]
create_empty = lambda s: np.array([[[np.nan] * 440] * 4] * s)

for site in ["LL", "LM", "LA", "CO", "HE"]:

    pdf1 = PdfPages(CONSTANTS.PLOT_PATH / f"xy_ridgeplots/{site}_distribution.pdf")
    pdf2 = PdfPages(CONSTANTS.PLOT_PATH / f"xy_ridgeplots/{site}_pixel_plot_normalized.pdf")
    pdf3 = PdfPages(CONSTANTS.PLOT_PATH / f"xy_ridgeplots/{site}_pixel_plot_unnormalized.pdf")

    tel_number = 3 if site == "HE" else 6
    data_raw = create_empty(tel_number)
    data_gain = create_empty(tel_number)
    data_lcu = create_empty(tel_number)
    data_ratio = create_empty(tel_number)
    data_old = create_empty(tel_number)
    data_gaetano = create_empty(tel_number)

    for i in range(tel_number):

        extension = site.lower() if site.lower() != "he" else f"he{i+1}"
        lcu_time = tools.pickle_load(f"/cr/tempdata01/filip/LCUDATA/data/x.{extension}")
        lcu_data = tools.pickle_load(f"/cr/tempdata01/filip/LCUDATA/data/y.{extension}")

        tel = XYScanner.Telescope(f"{site}{i+1}")

        for run in tel:
            if run.year_month_day.startswith("2019"): continue
            if run.source != "OLO": continue
            if run.stepsize != 6: continue
            if run.year_month_day.startswith("2024-12"): 
                run.year_month_day = "2024-11-30"
            j = campaign_dates.index(run.year_month_day[:-3])

            this_raw = run.get_data("XY")
            this_preXY = run.get_data("preXY")
            this_postXY = run.get_data("postXY")
            if this_raw is None \
            or this_preXY is None \
            or this_postXY is None: continue

            this_gain = this_raw / (0.5 * (this_preXY + this_postXY))
            this_ratio = run.get_data("ratio")
            this_old = this_ratio / this_gain

            # raise a warning if found closest index is last index
            # likely we need to get more data from lyon LCU files
            # don't forget to recreate {x,y}.* files via read_dat.ipynb
            run_time = datetime.strptime(run.year_month_day, "%Y-%m-%d")
            idx = tools.closest(lcu_time, run_time)

            if idx + 1 == len(lcu_time): raise ValueError(f"get more LCU data for {site = }!")
            this_lcu = this_raw / lcu_data[idx]
            
            data_raw[i, j] = this_raw
            data_gain[i, j] = this_gain
            data_lcu[i, j] = this_lcu
            data_ratio[i, j] = this_ratio
            data_old[i, j] = this_old
            data_gaetano[i, j] = this_ratio / lcu_data[idx]

    
    # raw data
    if True:
        raw = plot.ridgeplot2d(data_raw, 
                            cbar_ticks=campaign_dates, 
                            labels=[f"{site}{i}" for i in range(1, tel_number+1)],
                            xlabel=r"XY calib. / $\frac{\gamma}{\mathrm{ADC}}$",
                            ylabel="Campaign / YYYY-MM",)
        pdf1.savefig(raw)
        plt.close(raw)

        raw_pixels_normalized = pixel_grid(data_raw,
                                              xlabel=[f"{site}{i}" for i in range(1, tel_number+1)],
                                              ylabel=campaign_dates, vmin='min', vmax='max',
                                              cbar_label=r"XY calib. / $\frac{\gamma}{\mathrm{ADC}}$",)
        pdf2.savefig(raw_pixels_normalized)
        plt.close(raw_pixels_normalized)

        raw_pixels_unnormalized = pixel_grid(data_raw,
                                               xlabel=[f"{site}{i}" for i in range(1, tel_number+1)],
                                               ylabel=campaign_dates,
                                               cbar_label=r"XY calib. / $\frac{\gamma}{\mathrm{ADC}}$",)
        pdf3.savefig(raw_pixels_unnormalized)
        plt.close(raw_pixels_unnormalized)
        

    # gain
    if True:
        gain = plot.ridgeplot2d(data_gain * 1e6, 
                                cbar_ticks=campaign_dates, 
                                labels=[f"{site}{i}" for i in range(1, tel_number+1)],
                                xlabel=r"$10^6\times$XY calib. / FDEventSum",
                                ylabel="Campaign / YYYY-MM",)
        pdf1.savefig(gain)
        plt.close(gain)

        gain_pixels_normalized = pixel_grid(data_gain * 1e6, 
                                               xlabel=[f"{site}{i}" for i in range(1, tel_number+1)],
                                               ylabel=campaign_dates, vmin="min", vmax='max',
                                               cbar_label=r"$10^6\times$XY calib. / FDEventSum",)
        pdf2.savefig(gain_pixels_normalized)
        plt.close(gain_pixels_normalized)

        gain_pixels_unnormalized = pixel_grid(data_gain * 1e6, 
                                                 xlabel=[f"{site}{i}" for i in range(1, tel_number+1)],
                                                 ylabel=campaign_dates,
                                                 cbar_label=r"$10^6\times$XY calib. / FDEventSum",)
        pdf3.savefig(gain_pixels_unnormalized)
        plt.close(gain_pixels_unnormalized)


    # lcu
    if True:
        lcu = plot.ridgeplot2d(data_lcu * 1e6, 
                            cbar_ticks=campaign_dates, 
                            labels=[f"{site}{i}" for i in range(1, tel_number+1)],
                            xlabel=r"$10^6\times$XY calib. / LCU",
                            ylabel="Campaign / YYYY-MM",)
        pdf1.savefig(lcu)
        plt.close(lcu)

        lcu_pixels_normalized = pixel_grid(data_lcu * 1e6,
                                              xlabel=[f"{site}{i}" for i in range(1, tel_number+1)],
                                              ylabel=campaign_dates, vmin='min', vmax='max',
                                              cbar_label=r"$10^6\times$XY calib. / LCU",)
        pdf2.savefig(lcu_pixels_normalized)
        plt.close(lcu_pixels_normalized)

        lcu_pixels_unnormalized = pixel_grid(data_lcu * 1e6,
                                                xlabel=[f"{site}{i}" for i in range(1, tel_number+1)],
                                                ylabel=campaign_dates,
                                                cbar_label=r"$10^6\times$XY calib. / LCU",)
        pdf3.savefig(lcu_pixels_unnormalized)
        plt.close(lcu_pixels_unnormalized)


    # ratio
    if True:
        ratio = plot.ridgeplot2d(data_ratio, 
                                cbar_ticks=campaign_dates, 
                                labels=[f"{site}{i}" for i in range(1, tel_number+1)],
                                xlabel=r"XY calib. / std. calib.",
                                ylabel="Campaign / YYYY-MM",)
        pdf1.savefig(ratio)
        plt.close(ratio)

        ratio_pixels_normalized = pixel_grid(data_ratio,
                                                xlabel=[f"{site}{i}" for i in range(1, tel_number+1)],
                                                ylabel=campaign_dates, vmin="min", vmax="max", 
                                                cbar_label=r"XY calib. / std. calib.",)
        pdf2.savefig(ratio_pixels_normalized)
        plt.close(ratio_pixels_normalized)

        ratio_pixels_unnormalized = pixel_grid(data_ratio,
                                                  xlabel=[f"{site}{i}" for i in range(1, tel_number+1)],
                                                  ylabel=campaign_dates, 
                                                  cbar_label=r"XY calib. / std. calib.",)
        pdf3.savefig(ratio_pixels_unnormalized)
        plt.close(ratio_pixels_unnormalized)


    # old
    if True:
        old = plot.ridgeplot2d(data_old * 1e6, 
                            cbar_ticks=campaign_dates, 
                            labels=[f"{site}{i}" for i in range(1, tel_number+1)],
                            xlabel=r"$10^6$ XY calib. / (std. calib. $\times$ FDEventSum)",
                            ylabel="Campaign / YYYY-MM",)
        pdf1.savefig(old)
        plt.close(old)

        old_pixels_normalized = pixel_grid(data_old * 1e6,
                                              xlabel=[f"{site}{i}" for i in range(1, tel_number+1)],
                                              ylabel=campaign_dates, vmin="min", vmax="max", 
                                              cbar_label=r"$10^6$ XY calib. / (std. calib. $\times$ FDEventSum)",)
        pdf2.savefig(old_pixels_normalized)
        plt.close(old_pixels_normalized)

        old_pixels_unnormalized = pixel_grid(data_old * 1e6,
                                                xlabel=[f"{site}{i}" for i in range(1, tel_number+1)],
                                                ylabel=campaign_dates,
                                                cbar_label=r"$10^6$ XY calib. / (std. calib. $\times$ FDEventSum)",)
        pdf3.savefig(old_pixels_unnormalized)
        plt.close(old_pixels_unnormalized)


    # gaetano
    if True:
        gaetano = plot.ridgeplot2d(data_gaetano * 1e6, 
                                cbar_ticks=campaign_dates, 
                                labels=[f"{site}{i}" for i in range(1, tel_number+1)],
                                xlabel=r"$10^6 \times$ XY calib. / (std. calib. $\times$ LCU)",
                                ylabel="Campaign / YYYY-MM",)
        pdf1.savefig(gaetano)
        plt.close(gaetano)

        gaetano_pixels_normalized = pixel_grid(data_gaetano * 1e6, 
                                                  xlabel=[f"{site}{i}" for i in range(1, tel_number+1)],
                                                  ylabel=campaign_dates, vmin="min", vmax="max", 
                                                  cbar_label=r"$10^6 \times$ XY calib. / (std. calib. $\times$ LCU)")
        pdf2.savefig(gaetano_pixels_normalized)
        plt.close(gaetano_pixels_normalized)

        gaetano_pixels_unnormalized = pixel_grid(data_gaetano * 1e6, 
                                                    xlabel=[f"{site}{i}" for i in range(1, tel_number+1)],
                                                    ylabel=campaign_dates,
                                                    cbar_label=r"$10^6 \times$ XY calib. / (std. calib. $\times$ LCU)")
        pdf3.savefig(gaetano_pixels_unnormalized)
        plt.close(gaetano_pixels_unnormalized)

    pdf1.close()
    pdf2.close()
    pdf3.close()